In [8]:
# Load dataset
import netCDF4 as nc

# Open the file
ds = nc.Dataset('./power_901_annual_meteorology_utc.nc.nc4')

In [9]:
# Print the variables and dimensions in the file
print(ds.variables)

{'time': <class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: years since 1981-12-31 00:00:00
    long_name: time
unlimited dimensions: 
current shape = (2,)
filling on, default _FillValue of -2147483647 used, 'lat': <class 'netCDF4._netCDF4.Variable'>
float64 lat(lat)
    _FillValue: -999.0
    long_name: latitude
    standard_name: latitude
    units: degrees_north
unlimited dimensions: 
current shape = (361,)
filling on, 'lon': <class 'netCDF4._netCDF4.Variable'>
float64 lon(lon)
    _FillValue: -999.0
    long_name: longitude
    standard_name: longitude
    units: degrees_east
unlimited dimensions: 
current shape = (576,)
filling on, 'WS50M': <class 'netCDF4._netCDF4.Variable'>
float64 WS50M(time, lat, lon)
    _FillValue: -999.0
    long_name: Wind Speed at 50 Meters
    standard_name: Wind_Speed_at_50_Meters
    units: m/s
    valid_max: 75.0
    valid_min: 0.0
    valid_range: [ 0. 75.]
unlimited dimensions: 
current shape = (2, 361, 576)
filling on}


In [10]:
print(ds.dimensions)

{'time': <class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 2, 'lat': <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 361, 'lon': <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 576}


In [11]:
# print lowest and highest longitude and latitude
print(ds.variables['lon'][:].min(), ds.variables['lon'][:].max())
# print lowest and highest time
print(ds.variables['time'][:].min(), ds.variables['time'][:].max())

-180.0 179.375
38 39


In [12]:
from numpy.ma import masked

finalOutput = dict()

timeItems = ds.variables['time'][:]
windItems = ds.variables['WS50M'][:]
latItems = ds.variables['lat'][:]
lonItems = ds.variables['lon'][:]

for x in range(len(timeItems)):
    for y in range(len(latItems)):
        for z in range(len(lonItems)):
            val = windItems[x][y][z]
            if val is masked:
                continue

            lat = float(latItems[y])
            lon = float(lonItems[z])
            if lat not in finalOutput:
                finalOutput[lat] = dict()
            if lon not in finalOutput[lat]:
                finalOutput[lat][lon] = [0, 0]
            finalOutput[lat][lon][0] += val
            finalOutput[lat][lon][1] += 1
results = []
for lat in finalOutput:
    for lon in finalOutput[lat]:
        results.append([lat, lon, finalOutput[lat][lon][0] / finalOutput[lat][lon][1]])
results.sort()
with open("wind.csv", "w") as file:
    file.write(",".join(["lat", "lon", "WS50M"]))
    file.write("\n")
    for result in results:
        file.write(",".join([str(x) for x in result]))
        file.write("\n")            


In [1]:
import pandas as pd
import geopandas as gpd
windDf = pd.read_csv("wind.csv")
# https://stackoverflow.com/questions/56808444/reverse-geocoding-of-thousands-of-coordinates-to-get-country-only

gdf = gpd.GeoDataFrame(windDf, geometry=gpd.points_from_xy(windDf.lon, windDf.lat))
gdf.crs = "EPSG:4326"
# gdf = gdf.to_crs("EPSG:3857")
result = gpd.sjoin(gdf, gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')), how="left", op='intersects')
result
result.to_csv("wind_countries.csv", columns=["lat", "lon", "WS50M", "name", "iso_a3"])

c:\Users\Definitive\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
